# L'interrogation

In [3]:
%%shell
(cd samples/hellojpa && mvn --quiet clean package )

In [72]:
%%shell
docker start H2-JPADemo || docker run --quiet --detach --rm --name=H2-JPADemo \
    -e PUID=$UID -e PGID=$GID \
    -e H2_ARGS="-tcp -tcpAllowOthers -ifNotExists" \
    -p 9092:9092 \
    -v h2-data:/opt/h2-data \
    brunoe/docker-database-h2:develop

H2-JPADemo


In [20]:
%jars samples/hellojpa/target/*-SNAPSHOT-jar-with-dependencies.jar

import jakarta.persistence.*;
import fr.univtln.bruno.demos.jpa.hello.DatabaseManager;
System.setProperty("jakarta.persistence.jdbc.url","jdbc:h2:tcp://host.docker.internal/hellojpa-db");

try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) { }

In [4]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

Logger log = LoggerFactory.getLogger("Notebook");

Création de 1000 Customers aléatoires. 

In [22]:
import fr.univtln.bruno.demos.jpa.hello.samples.ex_entity.CustomerGenerator;

System.setProperty("jakarta.persistence.jdbc.url","jdbc:h2:tcp://host.docker.internal/hellojpa-db");

CustomerGenerator.generateCustomer(1000);

### SQL Natif

In [24]:
import jakarta.persistence.*;
import fr.univtln.bruno.demos.jpa.hello.DatabaseManager;
import fr.univtln.bruno.demos.jpa.hello.samples.ex_entity.Customer;

try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
    Query query = entityManager.createNativeQuery("""
                           SELECT * 
                           FROM CUSTOMER""",
                           Customer.class);
        
    //Customers (page 10 of size 5)
    int pageNumber = 10;
    int pageSize = 5;
    List<Customer> customers = query
        .setFirstResult((pageNumber-1)*pageSize)
        .setMaxResults(pageSize)
        .getResultList();
    
    customers.stream().forEach(System.out::println);
}    

Customer(creationDate=2023-03-22T18:20:59.861392, id=46, email=Drucilla.Crona@robel.io, firstname=Drucilla, lastname=Crona, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:20:59.864054, id=47, email=Stanford.Sipes@kilback.io, firstname=Stanford, lastname=Sipes, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:20:59.867097, id=48, email=Chau.Raynor@schuppe.com, firstname=Chau, lastname=Raynor, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:20:59.869775, id=49, email=Guy.Herzog@donnelly.com, firstname=Guy, lastname=Herzog, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:20:59.875923, id=50, email=Dave.Schmidt@bahringer.com, firstname=Dave, lastname=Schmidt, displayName=null, birthDate=null, photo=null, status=null)


In [25]:
try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
    long result = (Long) entityManager
                   .createNativeQuery("""
                                      SELECT COUNT(1)
                                      FROM CUSTOMER""")
                   .getSingleResult();
    System.out.println("Customers count: %s".formatted(result));
}

Customers count: 2000


In [26]:
String name = "Smith";
try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
    name = (String)entityManager.createNativeQuery("""
                SELECT LASTNAME FROM CUSTOMER  
                ORDER BY RAND ( )  
                LIMIT 1""").getSingleResult();
    }

In [27]:
//Variable name (String) contains an existing name 
try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
    Query query = entityManager.createNativeQuery("""
                                                  SELECT * 
                                                  FROM CUSTOMER
                                                  WHERE LASTNAME = ?
                                                  """, 
                                                  Customer.class);
    
    List<Customer> customers = query
        .setParameter(1, name)
        .setMaxResults(3)
        .getResultList();
    System.out.println("3 firsts customers whose name is %s".formatted(name));
    customers.stream().forEach(System.out::println);
}    

5 firsts customers whose name is Crooks
Customer(creationDate=2023-03-22T18:20:59.771941, id=18, email=Katie.Crooks@heller.org, firstname=Katie, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:21:00.491421, id=261, email=Gregorio.Crooks@lehner.net, firstname=Gregorio, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:21:00.953298, id=419, email=Grant.Crooks@mitchell.name, firstname=Grant, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:21:26.390721, id=1301, email=Benjamin.Crooks@cassin.org, firstname=Benjamin, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:21:26.547637, id=1366, email=Lida.Crooks@yundt.biz, firstname=Lida, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)


#### JPA Query Language (JPQL)

cf. [JPQL Wiki Book](https://en.wikibooks.org/wiki/Java_Persistence/JPQL)

In [36]:
EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()

In [37]:
//Variable name (String) contains an existing name 
TypedQuery<Customer> query = entityManager
        .createQuery("""
                     SELECT c
                     FROM Customer c
                     WHERE c.lastname = :name""",
                     Customer.class);

List<Customer> customers = query.setParameter("name", name)
    .setMaxResults(3)
    .getResultList();

customers.stream().forEach(System.out::println);

Customer(creationDate=2023-03-22T18:20:59.771941, id=18, email=Katie.Crooks@heller.org, firstname=Katie, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:21:00.491421, id=261, email=Gregorio.Crooks@lehner.net, firstname=Gregorio, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)
Customer(creationDate=2023-03-22T18:21:00.953298, id=419, email=Grant.Crooks@mitchell.name, firstname=Grant, lastname=Crooks, displayName=null, birthDate=null, photo=null, status=null)


```{literalinclude} samples/hellojpa/src/main/java/fr/univtln/bruno/demos/jpa/hello/samples/CustomerDisplayDTO.java
:language: java
:caption: CustomerDisplayDTO.java
:name: CustomerDisplayDTO.java
```

In [39]:
import fr.univtln.bruno.demos.jpa.hello.samples.ex_dto.CustomerDisplayDTO;

//Variable name (String) contains an existing name 

TypedQuery<CustomerDisplayDTO> query = entityManager.createQuery("""
                     SELECT new fr.univtln.bruno.demos.jpa.hello.samples.ex_dto.CustomerDisplayDTO
                         (c.firstname, c.lastname)
                     FROM Customer c
                     WHERE c.lastname = :name""",
                     CustomerDisplayDTO.class);

List<CustomerDisplayDTO> customers = query
        .setParameter("name", name)
        .setMaxResults(3)
        .getResultList();

customers.stream().forEach(System.out::println);

CustomerDisplayDTO[firstname=Katie, lastname=Crooks]
CustomerDisplayDTO[firstname=Gregorio, lastname=Crooks]
CustomerDisplayDTO[firstname=Grant, lastname=Crooks]


In [33]:
long customerCount=(long)entityManager.createQuery("""
                                SELECT COUNT(c)
                                FROM Customer c""").getSingleResult();
                                                   
System.out.println("There is %s customers.".formatted(customerCount));                                                   

There is 2000 customers.


In [34]:
entityManager.close()